In [2]:
import pandas as pd
import numpy as np
import requests

The dataset selected initially is "Air Quality Annual Summary" from ...

In [ ]:
!kaggle datasets download -d epa/air-quality

In [5]:
data_historic = pd.read_csv("C:\\Users\\Victo\\Downloads\\epa_air_quality_annual_summary.csv")
data_historic.head(5)

C:\Users\Victo\AppData\Local\Temp\ipykernel_4552\565861268.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_historic = pd.read_csv("C:\\Users\\Victo\\Downloads\\epa_air_quality_annual_summary.csv")


,state_code,county_code,site_num,parameter_code,poc,latitude,longitude,datum,parameter_name,sample_duration,...,seventy_five_percentile,fifty_percentile,ten_percentile,local_site_name,address,state_name,county_name,city_name,cbsa_name,date_of_last_change
0,20,57,1,82134,1,37.771964,-100.018190,WGS84,Molybdenum PM10 STP,24 HOUR,...,0.00000,0.0,0.0,DODGE CITY,PUMP STATION 2100 1ST AVE,Kansas,Ford,Dodge City,Dodge City KS,2012-08-11
1,18,163,12,42401,1,38.021727,-87.569458,WGS84,Sulfur dioxide,3-HR BLK AVG,...,3.60000,1.3,0.0,Evansville CAAP,425 WEST MILL ROAD/ FIRE STATION #17,Indiana,Vanderburgh,Evansville,Evansville IN-KY,2013-02-16
2,40,31,9000,88152,1,34.732404,-98.713351,WGS84,Phosphorus PM2.5 LC,24 HOUR,...,0.00000,0.0,0.0,NaN,Wichita Mountains,Oklahoma,Comanche,NaN,Lawton OK,2016-02-26
3,32,3,561,42601,1,36.163959,-115.113916,WGS84,Nitric oxide (NO),1 HOUR,...,18.80000,0.9,0.0,Sunrise Acres,2501 SUNRISE AVENUE,Nevada,Clark,Las Vegas,Las Vegas-Henderson-Paradise NV,2015-03-20
4,13,127,3001,12154,1,31.185187,-81.485166,WGS84,Selenium (TSP) STP,24 HOUR,...,0.00064,0.0,0.0,NaN,BRUNSWICK COASTAL COLLEGE,Georgia,Glynn,Brunswick,Brunswick GA,2015-07-22


In [7]:
data_historic.shape

(2038710, 55)

For possible machine learning tasks I decided to narrow down columns 

I selected



In [10]:
filtered_historic = data_historic[data_historic['parameter_name'] == 'Ozone'] 
filtered_historic.shape

(166005, 55)

Now I would use Air Quality System (AQS) API to enhance...

In [27]:
data_modern = data_historic.drop(data_historic.index)
data_modern = pd.DataFrame(columns=['state_code','county_code','site_number','parameter_code','poc','latitude','longitude', 'datum',
                                   'parameter', 'sample_duration_code', 'sample_duration', 'pollutant_standard', 'metric_used',
                                   'method', 'year', 'units_of_measure', 'event_type', 'observation_count', 'observation_percent',
                                   'validity_indicator', 'valid_day_count', 'required_day_count', 'exceptional_data_count',
                                   'null_observation_count', 'primary_exceedance_count', 'secondary_exceedance_count',
                                   'certification_indicator', 'arithmetic_mean', 'standard_deviation', 'first_max_value', 'first_max_datetime',
                                   'second_max_value', 'second_max_datetime', 'third_max_value', 'third_max_datetime', 'fourth_max_value',
                                   'fourth_max_datetime', 'first_max_nonoverlap_value', 'first_max_n_o_datetime', 'second_max_nonoverlap_value',
                                   'second_max_n_o_datetime', 'ninety_ninth_percentile', 'ninety_eighth_percentile', 'ninety_fifth_percentile',
                                   'ninetieth_percentile', 'seventy_fifth_percentile', 'fiftieth_percentile', 'tenth_percentile', 'local_site_name',
                                   'site_address', 'state', 'county', 'city', 'cbsa_code', 'cbsa', 'date_of_last_change'])


(0, 56)

In [11]:
key = 'sandgoose35'
email = 'VictorPakholkov@outlook.com'
base_url = 'https://aqs.epa.gov/data/api/annualData/'

In [19]:
params = {
    'email': email,
    'key': key,
    'param': '44201',  # Ozone parameter
    'bdate': '20180101',  # year 2018
    'edate': '20181212',  # year 2018
    'state': '01',  # state 01
}


In [20]:
response = requests.get(f'{base_url}byState', params=params)
if response.status_code == 200:
    data = response.json()
    for data_entry in data['Data']:
        data_to_append = pd.DataFrame([data_entry], columns=list(data_modern.columns))
        data_modern = pd.concat([data_modern, data_to_append], ignore_index=True)
else:
    print(f"API request failed with status code {response.status_code}")
    print(response.text)

TypeError: 'numpy.ndarray' object is not callable